# Exp4. 작사가 만들기

# 1. 데이터 읽어오기

In [1]:
import glob
import os
import numpy as np
import tensorflow as tf

txt_file_path = os.getenv('HOME') + '/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)    # glob 모듈의 glob 함수는 사용자가 제시한 조건에 맞는 파일명을 리스트로 반환

raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)
        
print("데이터 크기 : ", len(raw_corpus))
print("Examples : \n", raw_corpus[:3])

데이터 크기 :  187088
Examples : 
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


# 2. 데이터 정제

### 2-1. 길이가 0인 문장 제거

In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue
        
    if idx > 9: break     # 10개의 문장만 확인
        
    print(sentence)

Now I've heard there was a secret chord
That David played, and it pleased the Lord
But you don't really care for music, do you?
It goes like this
The fourth, the fifth
The minor fall, the major lift
The baffled king composing Hallelujah Hallelujah
Hallelujah
Hallelujah
Hallelujah Your faith was strong but you needed proof


### 2_2. 필터링 진행

In [3]:
import re

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()  # 소문자로 변경, 양쪽 공백 지우기
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)  # 특수문자 양쪽에 공백 넣기
    sentence = re.sub(r'[" "]+', " ", sentence)  # 여러개의 공백은 하나의 공백으로 변경
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # 괄호 안 특수문자가 아닌 모든 문자를 하나의 공백으로 변경
    sentence = sentence.strip()   # 양쪽 공백 지우기
    sentence = '<start> ' + sentence + ' <end>'
    return sentence

### 2-3. corpus 생성 (정규표현식 이용)

In [4]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
        
    preprocessed_sentence = preprocess_sentence(sentence)
    if len(preprocessed_sentence.split()) > 15: continue  # 토큰의 개수가 15개 넘어가는 문장 제외
        
    corpus.append(preprocessed_sentence)
    
corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

### 2-4. corpus를 tensor로 변환

In [5]:
# tokenizer는 문장으로부터 단어를 토큰화하고 숫자에 대응하는 딕셔너리를 사용할 수 있도록 함
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
    num_words = 12000, # 단어장의 크기는 12,000 이상으로 설정
    filters = ' ',     # 이미 문장을 정제했으니 filters 필요 없음
    oov_token = "<unk>"# 단어장에 포함되지 않은 단어를 '<unk>'로 변경
    )
    tokenizer.fit_on_texts(corpus)    # 문자 데이터를 입력받아 리스트 형태로 변환
    tensor = tokenizer.texts_to_sequences(corpus)   # 단어를 숫자의 시퀀스 형태로 변환
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding = 'post')   # 같은 길이의 시퀀스로 변환 (padding)
    
    print(tensor, tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    4 ...    0    0    0]
 [   2   15 2971 ...    0    0    0]
 [   2   33    7 ...   46    3    0]
 ...
 [   2    4  117 ...    0    0    0]
 [   2  258  195 ...   12    3    0]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fc0788c2ac0>


In [6]:
# tokenizer에 구축된 단어사전의 인덱스 확인

print(tensor[:3, :16])

[[   2   50    4   95  303   62   53    9  946 6269    3    0    0    0
     0]
 [   2   15 2971  872    5    8   11 5747    6  374    3    0    0    0
     0]
 [   2   33    7   40   16  164  288   28  333    5   48    7   46    3
     0]]


In [7]:
# tokenizer에 구축된 단어사전 내용 확인

for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])
    
    if idx >= 10:break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


# 3. 평가 데이터셋 분리

### 3-1.소스 문장 & 타겟 문장 생성

In [8]:
src_input = tensor[:, :-1] # 마지막 토큰 자름 (총 14개) → 소스 문장
tgt_input = tensor[:, 1:]  # 첫번째 토큰(<start>) 자름 (총 14개) → 타겟 문장

print(src_input[0])
print(tgt_input[0])

[   2   50    4   95  303   62   53    9  946 6269    3    0    0    0]
[  50    4   95  303   62   53    9  946 6269    3    0    0    0    0]


### 3-2. 훈련데이터와 평가데이터 분리

In [9]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size = 0.2, random_state = 42)
# 총 데이터의 20%를 평가 데이터셋으로 사용

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)
print("Source Val:", enc_val.shape)
print("Target Val:", dec_val.shape)

Source Train: (124981, 14)
Target Train: (124981, 14)
Source Val: (31246, 14)
Target Val: (31246, 14)


### 3-3. tf.data.Dataset 객체로 변환

In [10]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1 # 단어사전 12,000개 + <pad> = 12,001개

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)) # 데이터 셋 객체로 변환
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder = True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

# 4. 인공지능 만들기

In [11]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 518   # 워드 벡터의 차원수 (단어가 추상적으로 표현되는 크기)
hidden_size = 2048     # 모델에 얼마나 많은 일꾼을 둘 것인가?
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

# Embedding 레이어는 인덱스 값을 해당 인덱스번째의 워드 벡터로 바꿔줌

In [12]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법

for src_sample, tgt_sample in dataset.take(1): break
    
model(src_sample)

# 256 : 이전 스텝에서 지정한 배치 사이즈 (256개의 문장 데이터)
# 14 : LSTM은 자신에게 입력된 시퀀스의 길이만큼 동일한 길이의 시퀀스를 출력
# 12001 : Dence 레이어의 출력 차원수 (12001개의 단어)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 8.14897940e-05,  5.82887369e-05,  5.13098435e-04, ...,
          1.79299459e-04, -1.21782236e-04, -4.18903161e-04],
        [ 3.60988546e-04,  7.36650662e-04,  6.58627658e-04, ...,
          3.78477038e-04, -7.96392778e-05, -3.90958710e-04],
        [ 5.84731228e-04,  9.18104663e-04,  6.11621246e-04, ...,
          4.73846565e-04,  1.70839354e-04, -3.41477833e-04],
        ...,
        [ 1.45805941e-03,  2.40732334e-05, -6.18712947e-05, ...,
          1.60406227e-03,  1.64401915e-03,  6.37070392e-04],
        [ 1.38485013e-03, -1.92399253e-04, -3.04381101e-04, ...,
          1.91372214e-03,  2.01442582e-03,  5.95048303e-04],
        [ 1.34715601e-03, -4.28532541e-04, -4.95773915e-04, ...,
          2.29756301e-03,  2.32371432e-03,  4.79921087e-04]],

       [[ 8.14897940e-05,  5.82887369e-05,  5.13098435e-04, ...,
          1.79299459e-04, -1.21782236e-04, -4.18903161e-04],
        [ 2.22071933e-04, -1.12079186e-04,  8

In [13]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  6216518   
_________________________________________________________________
lstm (LSTM)                  multiple                  21028864  
_________________________________________________________________
lstm_1 (LSTM)                multiple                  33562624  
_________________________________________________________________
dense (Dense)                multiple                  24590049  
Total params: 85,398,055
Trainable params: 85,398,055
Non-trainable params: 0
_________________________________________________________________


# 5. 모델 학습시키기

* [조건] 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델 설계하기

[1차 시도]
* embedding_size = 256
* hidden_size = 1024
* bach_size = 256
* epochs = 10

loss : 2.20 / val_loss : 2.51

[2차 시도]
* embedding_size = 256
* hidden_size = 1024
* bach_size = 128
* epochs = 10

loss : 1.27 / val_loss : 2.44  
→ i love you , i m a liability

[3차 시도]
* embedding_size = 256
* hidden_size = 2048
* bach_size = 128
* epochs = 10

loss : 1.11 / val_loss : 2.24  
→ i love the way you lie  
▶ Epochs 9부터 val_loss가 다시 올리감

[4차 시도]
* embedding_size = 512
* hidden_size = 2048
* bach_size = 128
* epochs = 10

loss : 0.99 / val_loss : 2.24  
▶ Epoch 7부터 val_loss가 다시 올라감

[5차 시도]
* embedding_size = 512
* hidden_size = 2048
* bach_size = 128
* epochs = 8

loss : 1.05 / val_loss : 2.19 

In [14]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss = loss, optimizer = optimizer)
model.fit(enc_train, dec_train, validation_data=(enc_val, dec_val), epochs = 8, batch_size = 128)

Epoch 1/8
977/977 [==============================] - 284s 288ms/step - loss: 3.1092 - val_loss: 2.7864
Epoch 2/8
977/977 [==============================] - 291s 298ms/step - loss: 2.5643 - val_loss: 2.5040
Epoch 3/8
977/977 [==============================] - 292s 299ms/step - loss: 2.1661 - val_loss: 2.3231
Epoch 4/8
977/977 [==============================] - 292s 299ms/step - loss: 1.7989 - val_loss: 2.2111
Epoch 5/8
977/977 [==============================] - 292s 299ms/step - loss: 1.4962 - val_loss: 2.1545
Epoch 6/8
977/977 [==============================] - 292s 299ms/step - loss: 1.2742 - val_loss: 2.1423
Epoch 7/8
977/977 [==============================] - 292s 299ms/step - loss: 1.1301 - val_loss: 2.1689
Epoch 8/8
977/977 [==============================] - 292s 299ms/step - loss: 1.0526 - val_loss: 2.1944


# 6. 모델 평가하기

In [15]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):

    test_input = tokenizer.texts_to_sequences([init_sentence])  # 테스트를 위해 init_sentence도 텐서로 변환
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
        # 1
        predict = model(test_tensor)    # 입력받은 문장의 텐서 입력
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""

    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [16]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you so , hey <end> '

# 회고

### 1) 학습시간

이번 프로젝트를 하면서 다양한 시도를 하지 못한 것이 조금 아쉬웠다.  
학습시간이 너무나 오래 걸려서 이전 프로젝트와 같이 여러 시도를 해보지 못한 것이 아쉽다.  
텍스트 생성 모델을 학습시킬 때 원래 이렇게 학습시간이 오래 걸리는 것인지, 이것을 해결할 수 있는 방법이 있는지 궁금해졌다.

### 2) val_loss 맞추기

val_loss를 2.2 이하로 나오도록 맞추기 위해 다양한 시도를 하였다. 하이퍼파라미터값을 조정을 하였는데 확실히 val_loss 값이 내려가는 것을 확인할 수 있었다.  
하지만 epoch = 7 이후로는 내려갔던 val_loss 값이 다시 올라갔다. 그래서 embedding_size를 변경해서 다시 돌려보았으나 이 때도 똑같이 다시 올라간 것을 확인할 수 있었다.  
그래서 epoch를 줄이면 되지 않을까라고 생각을 해서 epoch를 8로 변경한 후 진행하였고, '2.19'라는 값을 얻게 되었다.  
사실 epoch 값을 줄이지 않고 다른 하이퍼 파라미터를 조정하면 되었을 수 있었는데, 제출 기한이 있다보니 여러 시도를 해보지 못하였다.

### 3) 생성된 가사

사실 val_loss를 맞추는 것에 너무나 초점을 두어 돌릴 때마다 생성된 문장을 몇 개만 확인하였다.  
val_loss가 2.2 이하로 내려갔을 때 나온 문장은 'i love you so , hey'이다. 조금 더 다양한 단어를 사용한 문장이 나왔으면 하는 바람이 있어서 그런지 생성된 문장은 다소 아쉬웠다.  
val_loss가 2.4 정도 되었을 때 생성된 문장은 아래와 같다.
*  i love you , i m a liability
* i love the way you lie  

첫번째 문장을 보면 val_loss가 높아도 괜찮은 문장이 생성되었다. 매우 사랑꾼 같은 문장이 생성되었다.  
하지만 두번째 문장은 조금 부정적인 느낌이 들었다. hidden_size를 2배로 높여서 이런 문제가 발생하는 건가라는 생각이 들었다.  

▶ val_loss를 맞추는 것도 중요하지만 어떠한 문장이 생성되었는지, 문맥에 잘 맞는 문장인지, 또 예상과 다르게 부정적인 문장인지 확인을 해서 하이퍼파라미터 등 다양한 방법으로 모델을 구축하는 것도 중요하다고 생각했다.

### 4) 추가로 시도해 볼 것

* epoch 값을 10으로 유지한채로 다른 하이퍼파라미터를 조정하였을 때 val_loss 값을 2.2로 맞추는 방법
* hidden_size에 따라 문장이 어떻게 생성되는지 확인하기